In [1]:
! pip -q install langchain openai

In [2]:
! pip show langchain

Name: langchain
Version: 0.3.23
Summary: Building applications with LLMs through composability
Home-page: 
Author: 
Author-email: 
License: MIT
Location: /home/anudeep/miniconda3/envs/genai/lib/python3.11/site-packages
Requires: langchain-core, langchain-text-splitters, langsmith, pydantic, PyYAML, requests, SQLAlchemy
Required-by: langchain-community


In [3]:
# load pdf file
import pandas as pd

df = pd.read_csv('/home/anudeep/Desktop/gen ai/document/mtsamples.csv')

In [4]:
df

,Unnamed: 0,description,medical_specialty,sample_name,transcription,keywords
0,0,A 23-year-old white female presents with comp...,Allergy / Immunology,Allergic Rhinitis,"SUBJECTIVE:, This 23-year-old white female pr...","allergy / immunology, allergic rhinitis, aller..."
1,1,Consult for laparoscopic gastric bypass.,Bariatrics,Laparoscopic Gastric Bypass Consult - 2,"PAST MEDICAL HISTORY:, He has difficulty climb...","bariatrics, laparoscopic gastric bypass, weigh..."
2,2,Consult for laparoscopic gastric bypass.,Bariatrics,Laparoscopic Gastric Bypass Consult - 1,"HISTORY OF PRESENT ILLNESS: , I have seen ABC ...","bariatrics, laparoscopic gastric bypass, heart..."
3,3,2-D M-Mode. Doppler.,Cardiovascular / Pulmonary,2-D Echocardiogram - 1,"2-D M-MODE: , ,1. Left atrial enlargement wit...","cardiovascular / pulmonary, 2-d m-mode, dopple..."
4,4,2-D Echocardiogram,Cardiovascular / Pulmonary,2-D Echocardiogram - 2,1. The left ventricular cavity size and wall ...,"cardiovascular / pulmonary, 2-d, doppler, echo..."
...,...,...,...,...,...,...
4994,4994,Patient having severe sinusitis about two to ...,Allergy / Immunology,Chronic Sinusitis,"HISTORY:, I had the pleasure of meeting and e...",NaN
4995,4995,This is a 14-month-old baby boy Caucasian who...,Allergy / Immunology,Kawasaki Disease - Discharge Summary,"ADMITTING DIAGNOSIS: , Kawasaki disease.,DISCH...","allergy / immunology, mucous membranes, conjun..."
4996,4996,A female for a complete physical and follow u...,Allergy / Immunology,Followup on Asthma,"SUBJECTIVE: , This is a 42-year-old white fema...",NaN
4997,4997,Mother states he has been wheezing and coughing.,Allergy / Immunology,Asthma in a 5-year-old,"CHIEF COMPLAINT: , This 5-year-old male presen...",NaN


In [5]:
! pip install langchain_experimental

In [6]:
from langchain_experimental.agents import create_csv_agent
from langchain.llms import OpenAI
! pip install tabulate


In [9]:
from langchain_experimental.agents import create_csv_agent
from langchain.chat_models import ChatOpenAI
from langchain.agents import AgentType 
import os
from dotenv import load_dotenv

# Load the .env file
load_dotenv()

# Get the key from env
api_key = os.getenv("OPENAI_KEY")

# Initialize LLM
llm = ChatOpenAI(temperature=0, model="gpt-4o-mini", openai_api_key=api_key)

# Create the agent
agent = create_csv_agent(
    llm=llm,
    path="/home/anudeep/Desktop/gen ai/document/mtsamples.csv",  
    verbose=True,
    agent_type=AgentType.OPENAI_FUNCTIONS,
    allow_dangerous_code=True
)

# Test the agent
response = agent.run("Summarize the data")
print(response)


/tmp/ipykernel_1357344/3798045111.py:26: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = agent.run("Summarize the data")




> Entering new AgentExecutor chain...
The dataframe `df` contains medical consultation records with the following columns:

1. **Unnamed: 0**: An index column.
2. **description**: A brief description of the patient's condition or the purpose of the consultation.
3. **medical_specialty**: The medical specialty relevant to the consultation (e.g., Allergy / Immunology, Bariatrics, Cardiovascular / Pulmonary).
4. **sample_name**: A title or identifier for the specific consultation record.
5. **transcription**: A detailed transcription of the patient's medical history, examination findings, assessments, and plans.
6. **keywords**: A list of keywords associated with the consultation, summarizing key topics and conditions mentioned in the transcription.

### Summary of the Data:
- The dataset includes various medical specialties, primarily focusing on conditions related to allergies, bariatric surgery, and cardiovascular health.
- Each record provides a comprehensive overview of the patient

In [10]:
agent.run("how many medical specialty has the uniqueness ")



> Entering new AgentExecutor chain...

Invoking: `python_repl_ast` with `{'query': "df['medical_specialty'].nunique()"}`


40There are 40 unique medical specialties in the dataframe.

> Finished chain.


'There are 40 unique medical specialties in the dataframe.'

In [11]:
df['medical_specialty'].nunique()

40

In [12]:
# For second dataset  
from datasets import load_dataset

# Load the dataset
dataset = load_dataset("rungalileo/medical_transcription_40")


/home/anudeep/miniconda3/envs/genai/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [13]:
# Set the format to pandas DataFrame
dataset.set_format(type='pandas')

# Convert the training data to a pandas DataFrame
train_df = dataset['train'][:]

# Save the DataFrame to a CSV file
train_df.to_csv("medical_transcription_train.csv", index=False)

# Reset the dataset format
dataset.reset_format()


In [15]:
from langchain_experimental.agents import create_csv_agent
from langchain.chat_models import ChatOpenAI
from langchain.agents import AgentType  

# Initialize LLM
llm = ChatOpenAI(temperature=0, model="gpt-4o-mini", openai_api_key=api_key)

# Create the agent
agent = create_csv_agent(
    llm=llm,
    path="medical_transcription_train.csv",  
    verbose=True,
    agent_type=AgentType.OPENAI_FUNCTIONS,
    allow_dangerous_code=True
)

# Test the agent
response = agent.run("Summarize the data")
print(response)



> Entering new AgentExecutor chain...

Invoking: `python_repl_ast` with `{'query': "df.describe(include='all')"}`


                 id                                               text  \
count   4499.000000                                               4470   
unique          NaN                                               3966   
top             NaN  REASON FOR CONSULTATION: , Thyroid mass diagno...   
freq            NaN                                                  4   
mean    2489.084019                                                NaN   
std     1445.637607                                                NaN   
min        0.000000                                                NaN   
25%     1238.500000                                                NaN   
50%     2489.000000                                                NaN   
75%     3740.500000                                                NaN   
max     4998.000000                                                N

In [16]:
print(response)

The dataset contains 4,499 entries with the following key characteristics:

1. **ID**:
   - The `id` column ranges from 0 to 4998.
   - The mean ID is approximately 2489, with a standard deviation of about 1445.
   - The IDs are unique, with a total of 4,499 unique values.

2. **Text**:
   - The `text` column contains 4,470 entries, with a total of 3,966 unique texts.
   - The most frequently occurring text (top) appears 4 times, indicating some repetition in the dataset.

3. **Label**:
   - The `label` column has 4,499 entries, with a mean value of approximately 21.5 and a standard deviation of about 10.3.
   - The labels range from a minimum of 0 to a maximum of 39.
   - The 25th percentile of labels is 13, the 50th percentile (median) is 25, and the 75th percentile is 29.

Overall, the dataset appears to be a collection of medical texts with associated labels, likely indicating some classification or categorization of the text content.
